<a href="https://colab.research.google.com/github/huyenntt1234/MLProject-Child_Mindset/blob/main/MLProject_Childhood.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

nghiemhuyen_child_mindset_institution_path = kagglehub.dataset_download('nghiemhuyen/child-mindset-institution')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.base import clone
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import StratifiedKFold
from scipy.optimize import minimize
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
from matplotlib.ticker import MaxNLocator, FormatStrFormatter, PercentFormatter
from sklearn.preprocessing import StandardScaler
from keras.models import Model
from keras.layers import Input, Dense
from keras.optimizers import Adam
from colorama import Fore, Style
from IPython.display import clear_output
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import VotingRegressor, RandomForestRegressor, GradientBoostingRegressor
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
train=pd.read_csv('/kaggle/input/child-mindset-institution/train.csv')
test=pd.read_csv('/kaggle/input/child-mindset-institution/test.csv')
sample=pd.read_csv('/kaggle/input/child-mindset-institution/sample_submission.csv')
sii_labels =['None','Mild','Moderate','Severe']

In [ ]:
train.describe()

In [ ]:
train_missing=train.isnull().sum()
train_not_missing=train.count(numeric_only=False)
train_missing=train_missing/(train_missing+train_not_missing)
train_missing=train_missing[train_missing>0.2]
train_missing=train_missing.to_frame()
train_missing.describe()
#74 features have more than 20% of missing values in the training data. The max % of missing value is 88%.
#Therefore, the train data is needed to fill missing value before processing.

In [ ]:
 train.groupby('sii').size()
#sii,the targeted value, is not evenly distributed in the train data.

In [ ]:
test.describe()
# there are some features in train data that are missing in test data.

In [ ]:
features_not_in_test=[f for f in train.columns if f not in test.columns]
features_not_in_test.remove('sii')
train=train.drop(features_not_in_test,axis=1)
train.describe()

In [ ]:
#Demographic info about the train data
#Enrollment by seasons is evently
vc=train['Basic_Demos-Enroll_Season'].value_counts()
vc=vc.to_frame(name="Count")
vc['index']=vc.index
plt.pie(vc['Count'],labels=vc['index'],autopct = "%.2f%%")
plt.title('Season of enrollment')
plt.show()


In [ ]:
# Participants are more girls than boys
vc=train['Basic_Demos-Sex'].value_counts()
vc=vc.to_frame(name="Count")
plt.pie(vc['Count'],labels=['Girls','Boys'],autopct = "%.2f%%",colors=["lightpink", "lightblue"])
plt.title('Participants by genders')
plt.show()

In [ ]:
# Age distribution by genders is quite similar with a right skew shape. In both genders, mode of age is 8.
vc = train.get(['Basic_Demos-Sex','Basic_Demos-Age'])
vc = vc.groupby(['Basic_Demos-Sex']).value_counts()
vc = vc.to_frame(name='counts')
for i in vc.index.names:
    vc[i] = vc.index.get_level_values(i)
_, axs = plt.subplots(2, 1, sharex=True)
for sex in range(2):
    ax = axs.ravel()[sex]
    ax.bar(vc['Basic_Demos-Age'],
           vc['counts'],
           color=['lightpink', 'lightblue'][sex],
           label=['Girls', 'Boys'][sex])
    ax.xaxis.set_major_locator(MaxNLocator(integer=True))
    ax.set_ylabel('count')
    ax.legend()
plt.suptitle('Age distribution')
axs.ravel()[1].set_xlabel('years')
plt.show()

In [ ]:
#sii distribution by genders is quite similar with a right skew shape. In both genders, mode of sii label is none.
vc = train.get(['Basic_Demos-Sex','sii'])
vc = vc.groupby(['Basic_Demos-Sex']).value_counts()
vc = vc.to_frame(name='counts')
for i in vc.index.names:
    vc[i] = vc.index.get_level_values(i)
_, axs = plt.subplots(2, 1, sharex=True)
for sex in range(2):
    ax = axs.ravel()[sex]
    ax.bar(vc['sii'],
           vc['counts']/vc['counts'].sum(),
           color=['lightpink', 'lightblue'][sex],
           label=['Girls', 'Boys'][sex])
    ax.set_xticks(np.arange(4),sii_labels)
    ax.yaxis.set_major_formatter(PercentFormatter(xmax=1,decimals=0))
    ax.set_ylabel('count')
    ax.legend()
plt.suptitle('sii distribution')
axs.ravel()[1].set_xlabel('sii lable')
plt.show()

KNN

In [ ]:
imputer=KNNImputer(n_neighbors=5)
numeric_cols = train.select_dtypes(include=['float64','int64']).columns
imputed_data=imputer.fit_transform(train[numeric_cols])
train_imputed=pd.DataFrame(imputed_data,columns=numeric_cols)

train_imputed['sii']=train_imputed['sii'].round().astype(int)
for col in train.columns:
    if col not in numeric_cols:
        train_imputed[col]=train[col]

train=train_imputed

Feature engineering

In [ ]:
def feature_engineering(df):
    numerics=['int16','int32','int64','float16','float32','float64']
    numeric_df=df.select_dtypes(include=numerics).copy()
    Physical_Season=df['Physical-Season']
    Physical_Season = pd.get_dummies(Physical_Season).astype(int)
    SDS_Season=df['SDS-Season']
    SDS_Season = pd.get_dummies(SDS_Season).astype(int).rename(columns=lambda x:'SDS_' +str(x))
    df=pd.concat([numeric_df,Physical_Season,SDS_Season],axis=1)
    df['BMI_Age']=df['Physical-BMI']/df['Basic_Demos-Age']
    df['Internet_Hours_Age']=df['PreInt_EduHx-computerinternet_hoursday']*df['Basic_Demos-Age']
    df['PAQ_A_BMI']=df['PAQ_A-PAQ_A_Total']/df['BIA-BIA_BMI']
    df['PAQ_C_BMI']=df['PAQ_C-PAQ_C_Total']/df['BIA-BIA_BMI']
    df['SDS_Age']=df['SDS-SDS_Total_T']*df['Basic_Demos-Age']
    return df


In [ ]:
train=feature_engineering(train)
test=feature_engineering(test)

In [ ]:
def quadratic_weight_kappa(y_true,y_pred):
    return cohen_kappa_score(y_true,y_pred,weights='quadratic')
def threshold_Rounder(oof_non_rounded,thresholds):
    return np.where(oof_non_rounded<thresholds[0],0,
                   np.where(oof_non_rounded<thresholds[1],1,
                           np.where(oof_non_rounded<thresholds[2],2,3)))
def evaluate_prediction(thresholds, y_true,oof_non_rounded):
    rounded_p=threshold_Rounder(oof_non_rounded,thresholds)
    return -quadratic_weight_kappa(y_true,rounded_p)

Model

In [ ]:
SEED=42
n_splits=5

In [ ]:
def TrainML(model_class,test_data):
    X=train.drop(['sii'],axis=1)
    y=train['sii']

    SKF=StratifiedKFold(n_splits=n_splits,shuffle=True,random_state=SEED)

    train_S=[]
    test_S=[]

    oof_non_rounded=np.zeros(len(y),dtype=float)
    oof_rounded=np.zeros(len(y),dtype=int)
    test_preds = np.zeros((len(test_data),n_splits))

    for fold, (train_idx,test_idx)in enumerate(tqdm(SKF.split(X,y),desc = "Training Folds",total=n_splits)):
       X_train, X_test=X.iloc[train_idx],X.iloc[test_idx]
       y_train, y_test=y.iloc[train_idx],y.iloc[test_idx]

       model=clone(model_class)
       model.fit(X_train,y_train)

       y_train_pred=model.predict(X_train)
       y_test_pred=model.predict(X_test)

       oof_non_rounded[test_idx]=y_test_pred
       y_test_pred_rounded = y_test_pred.round(0).astype(int)
       oof_rounded[test_idx]=y_test_pred_rounded

       train_kappa=quadratic_weight_kappa(y_train,y_train_pred.round(0).astype(int))
       test_kappa=quadratic_weight_kappa(y_test,y_test_pred_rounded)

       train_S.append(train_kappa)
       test_S.append(test_kappa)

       test_preds[:,fold]=model.predict(test_data)

       print(f"Fold{fold+1}-TrainQWK:{train_kappa:.4f},Validation QWK:{test_kappa:.4f}")
       clear_output(wait=True)

    print(f"Mean Train QWK -->{np.mean(train_S):.4f}")
    print(f"Mean Validation QWK -->{np.mean(test_S):.4f}")

    KappaOPtimizer=minimize(evaluate_prediction,x0=[0.5,1.5,2.5],args=(y,oof_non_rounded),method='Nelder-Mead')
    assert KappaOPtimizer.success, "Optimization did not converge."

    oof_tuned=threshold_Rounder(oof_non_rounded,KappaOPtimizer.x)
    tKappa=quadratic_weight_kappa(y,oof_tuned)

    print(f"---->||Optimized QMK SCORE::{tKappa:.3f}")

    tpm=test_preds.mean(axis=1)
    tpTuned=threshold_Rounder(tpm,KappaOPtimizer.x)

    submission=pd.DataFrame({'id':sample['id'],'sii':tpTuned})
    return submission

In [ ]:
#Model parameter:
#Light Gradient Boosting Regressor
LGBM_params={
    'n_estimators': 200,
    'learning_rate': 0.05,
    'max_depth': 12,
    'feature_fraction':0.8,
    'bagging_fraction':0.8,
    'bagging_freq':4,
    'lamda_l1':8,
    'lamda_l2':0.05,
   }
#XGBoost:
XGB_params={
    'n_estimators': 200,
    'learning_rate': 0.05,
    'max_depth': 12,
    'subsample':0.5,
    'colsample_bytree': 0.5,
    'reg_alpha': 0.5,
    'reg_lambda': 1,
    'random_state':SEED
}

In [ ]:
Light=LGBMRegressor(**LGBM_params,random_state=SEED,verbose=-1)
XGB_model=XGBRegressor(**XGB_params)

In [ ]:
voting_model=VotingRegressor(estimators=[('lightgbm',Light),('xgboost',XGB_model)])
submission = TrainML(voting_model,test)

Hyperparameter Tuning - Randomized Search CV

In [ ]:
# Light Gradient Boosting Regressor
LGBM_param_dist = {
    'n_estimators': np.arange(200,500,50),
    'learning_rate': np.linspace(0.01,0.1,10),
    'max_depth': np.arange(4,14,2),
    'feature_fraction':np.linspace(0.5,1.0,10),
    'bagging_fraction':np.linspace(0.5,1.0,10),
    'bagging_freq':np.arange(2,10,2),
    'lamda_l1':np.arange(5,15,2),
    'lamda_l2':np.linspace(0.01,1.0,10),
   }
#XGBoost:
XGB_param_dist={
    'n_estimators': np.arange(200,500,50),
    'learning_rate': np.linspace(0.01,0.1,10),
    'max_depth': np.arange(4,10,2),
    'subsample':np.linspace(0.5,1.0,10),
    'colsample_bytree':np.linspace(0.5,1.0,10),
    'reg_alpha':np.linspace(0.5,1.5,10),
    'reg_lambda':np.linspace(1,10,10),
    }


In [ ]:
#X_train, y_train
X_train=train.drop(['sii'],axis=1)
y_train=train['sii']

In [ ]:
#LBGM
LBGM_search=RandomizedSearchCV(Light,LGBM_param_dist,n_iter=20,scoring='neg_mean_squared_error',cv=3,random_state=SEED)
LBGM_search.fit(X_train,y_train)
best_LGBM=LBGM_search.best_estimator_
#XGB
XGB_search=RandomizedSearchCV(XGB_model,XGB_param_dist,n_iter=20,scoring='neg_mean_squared_error',cv=3,random_state=SEED)
XGB_search.fit(X_train,y_train)
best_XGB=XGB_search.best_estimator_


In [ ]:
best_LGBM

In [ ]:
best_XGB

In [ ]:
#Update hyperparameters
LGBM_params={
    'n_estimators': 250,
    'learning_rate': 0.02,
    'max_depth': 8,
    'feature_fraction':0.611,
    'bagging_fraction':0.556,
    'bagging_freq':4,
    'lamda_l1':11,
    'lamda_l2':1.0,
   }
#XGBoost:
XGB_params={
    'n_estimators': 350,
    'learning_rate': 0.02,
    'max_depth': 8,
    'subsample':0.5,
    'colsample_bytree': 0.5,
    'reg_alpha': 0.5,
    'reg_lambda': 1,
    'random_state':SEED
}

Light=LGBMRegressor(**LGBM_params,random_state=SEED,verbose=-1)
XGB_model=XGBRegressor(**XGB_params)
voting_model=VotingRegressor(estimators=[('lightgbm',Light),('xgboost',XGB_model)])
submission1 = TrainML(voting_model,test)